In [1]:
cd /content/drive/MyDrive/PoliTo/NLP_Polito/progetto/codice/CharBERT

/content/drive/MyDrive/PoliTo/NLP_Polito/progetto/codice/CharBERT


In [2]:
%%capture
%pip install boto3 sentence_transformers datasets

In [3]:
from modeling.modeling_charbert import CharBertTransformer
from processors.utils_emb import emb_convert_examples_to_features
from time import time
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from tqdm import tqdm

In [28]:
from datasets import load_dataset
dataset = load_dataset("vishnupriyavr/wiki-movie-plots-with-summaries")
sentences = dataset['train']['Plot']
num_sent = 2000
sentences = sentences[1000:num_sent]

## Charbert Time

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
cbt = CharBertTransformer(model_type = 'bert',
                        model_name_or_path = '/content/drive/MyDrive/PoliTo/NLP_Polito/progetto/codice/CharBERT/models/charbert-bert-wiki',
                        char_vocab = '/content/drive/MyDrive/PoliTo/NLP_Polito/progetto/codice/CharBERT/data/dict/bert_char_vocab'
                        )
cbt.eval()
cbt.to(device)

cls: <class 'modeling.configuration_bert.BertConfig'>
pretrained_model_name_or_path: /content/drive/MyDrive/PoliTo/NLP_Polito/progetto/codice/CharBERT/models/charbert-bert-wiki
cls.pretrained_config_archive_map: {'bert-base-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json', 'bert-large-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json', 'bert-base-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json', 'bert-large-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-config.json', 'bert-base-multilingual-uncased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json', 'bert-base-multilingual-cased': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', 'bert-base-chinese': 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json', '

Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: CharBertModel 

In [36]:
tok_time_start = time()

output = cbt.tokenize(sentences)

tok_time_end = time()
tok_time = tok_time_end - tok_time_start

print(f"Charbert tokenization time: {tok_time:2f} sec")

total_tokens = output['attention_mask'].sum()

Charbert tokenization time: 6.842124 sec


In [37]:
list_output = []
for k, v in output.items():
    print(k)
    list_output.append(v)

char_input_ids
start_ids
end_ids
input_ids
attention_mask
token_type_ids


In [38]:
eval_batch_size = 4
enc_time_start = time()
eval_dataset = TensorDataset(*list_output)

eval_sampler = SequentialSampler(eval_dataset)
eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size)

for batch in tqdm(eval_dataloader, desc="Running..."):
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {'char_input_ids': batch[0],
                'start_ids':      batch[1],
                'end_ids':        batch[2],
                'input_ids':      batch[3],
                'attention_mask': batch[4],
                'token_type_ids': batch[5]}

    outputs = cbt(inputs)

enc_time_end = time()
enc_time = enc_time_end - enc_time_start
print(f"Charbert encoding time: {enc_time:.2f} sec")

Running...: 100%|██████████| 250/250 [01:39<00:00,  2.51it/s]

Charbert encoding time: 99.56 sec


In [39]:
print(tok_time)
print (enc_time)

6.8421244621276855
99.55968236923218


In [35]:
total_tokens.item()

305800

In [12]:
print(f'Tokenization: {(total_tokens/tok_time):.2f} token per sec')
print(f'Encoding: {(total_tokens/enc_time):.2f} token per sec')
print(f'Total: {(total_tokens/(tok_time + enc_time)):.2f} token per sec')

Tokenization: 44277.55 token per sec
Encoding: 2482.05 token per sec
Total: 2350.30 token per sec


In [13]:
tot_sent = len(sentences)

print(f'Tokenization: {((tot_sent * 512)/tok_time):.2f} token per sec')
print(f'Encoding: {((tot_sent * 512)/enc_time):.2f} token per sec')
print(f'Total: {((tot_sent * 512)/(tok_time + enc_time)):.2f} token per sec')

Tokenization: 89934.69 token per sec
Encoding: 5041.43 token per sec
Total: 4773.83 token per sec
